# Тестовое задание Product Analyst
---

В продукте Stoic решили провести эксперимент, призванный улучшить конверсию C1 (sign-up->purchase) в мобильных приложениях Stoic на обеих платформах. Для этого в приложении изменили порядок шагов в воронке, поставив шаг воронки «платеж» раньше, чем шаг воронки «привязка кошелька». Версия появилась в сторах 19.12.2020.

Файл с данными размещен по [ссылке](https://docs.google.com/spreadsheets/d/1p7I1lresp_R6BrT-HqnLn5Og9joQW86Qq__Vsd5Fe2s/edit?usp=sharing). 

Задачи:



1.   Используя данные выгрузок из внутренней платежной системы (payments) и из аналитической системы (sign-ups), определите, получилось ли добиться цели эксперимента, произведя необходимые расчеты и описав методологию расчётов и выводы в документе. Для решения задачи можно использовать любой аналитический пакет (Python, R, Excel, Spreadsheets etc), приложите код/файл с расчетами вместе с документом с выводами.
2.   Если возможно, определите наиболее эффективные каналы привлечения и другие важные параметры юнит-экономики продукта, которые заметите на основе данных из п.1.
3.   Определите потенциальные проблемные зоны в механизмах сбора данных.









## Ипорт и загрузка данных


In [ ]:
import pandas as pd
import numpy as np
import math as mth
import scipy.stats as st

import requests

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# from pymystem3 import Mystem
# from collections import Counter

In [ ]:
# Загружаем в DataFrame данные из листа payments
sheet_url = 'https://docs.google.com/spreadsheets/d/1p7I1lresp_R6BrT-HqnLn5Og9joQW86Qq__Vsd5Fe2s/edit#gid=0'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
payments = pd.read_csv(csv_export_url, sep=',')

# Немного красоты и удобства в заголовках 
payments.columns = payments.columns.str.lower()
payments.rename(columns={'payment dt': 'payment_dt'}, inplace=True)

# payments.columns = payments.columns.str.lower()
payments.head()

,email,amount,balance,payment_dt
0,petersciolto@gmail.com,50.0,1000.0,1/10/2021 23:13:38
1,chappelusa@hotmail.com,50.0,1000.0,1/10/2021 21:28:32
2,brianbutcher@mail.com,50.0,1000.0,1/10/2021 19:19:40
3,svarmuza@gmail.com,50.0,1000.0,1/10/2021 18:47:52
4,chalotteframe@icloud.com,50.0,1000.0,1/10/2021 18:34:19


In [ ]:
# Загружаем в DataFrame данные из листа sign-ups
sheet_url = 'https://docs.google.com/spreadsheets/d/1p7I1lresp_R6BrT-HqnLn5Og9joQW86Qq__Vsd5Fe2s/edit#gid=347578162'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')               
sign_ups = pd.read_csv(csv_export_url, sep=',', skipinitialspace=True)
sign_ups.head()

,\tamplitude_id,\tcarrier,\tcity,\tcountry,\tdevice,\tdevice_type,\tdma,\tip_address,\tlanguage,\tlibrary,\tos,\tplatform,\tregion,\tstart_version,\tversion,\tgp:CAPMAIGN,\tgp:COHORT_DAY,\tgp:COHORT_MONTH,\tgp:COHORT_WEEK,\tgp:COHORT_YEAR,\tgp:IS_DEMO,gp:MEDIA_SOURCE,\tgp:USER_ID
0,41155702895,\tBell,\tLaval,\tCanada,\tApple iPhone,\tApple iPhone 8,\t,\t192.222.239.121,\tEnglish,\tamplitude-ios/4.10.0,\tios 13.4.1,\tiOS,\tQuebec,16.0,16.0,\tApple Store Website Link,6,1,2,2021,\t,\tCindicator Website,\tinsidebar@protonmail.com
1,42912229947,\tTele2,\tMoscow,\tRussia,\tApple iPhone,\tiPhone 11 Pro,\t,\t176.59.43.9,\tEnglish,\tamplitude-ios/4.10.0,\tios 14.1,\tiOS,\tMoscow,16.0,16.0,\t,1,1,53,2021,\tTrue,\t,\ttalkshowhost23@gmail.com
2,159955861327,\tAT&T,\tBradenton,\tUnited States,\tApple iPhone,\tiPhone XR,"\tTampa-St Petersburg (Sarasota), FL",\t96.59.200.237,\tEnglish,\tamplitude-ios/4.10.0,\tios 14.2,\tiOS,\tFlorida,16.0,16.0,\tCindicator Website QR,6,1,2,2021,\tTrue,\tQR_code,\tnumanmete@gmail.com
3,221417831075,\tVerizon,\tColumbus,\tUnited States,\tApple iPhone,\tApple iPhone 12 Mini,"\tColumbus, OH",\t66.115.189.206,\tEnglish,\tamplitude-ios/4.10.0,\tios 14.2.1,\tiOS,\tOhio,16.0,16.0,\tApple Store Website Link,1,1,1,2021,\tTrue,\tCindicator Website,\tshaunlentini@gmail.com
4,221439782047,\tTelstra,\tMelbourne,\tAustralia,\tSamsung Galaxy Note,\tSamsung Galaxy Note 10+,\t,\t116.255.20.90,\tEnglish,\tamplitude-android/2.25.2,\tandroid 10,\tAndroid,\tVictoria,19.0,19.0,\t,\t,\t,\t,\t,\tTrue,\t,\tnajmulq@gmail.com


А вот и первая потенциальная проблема сбора данных - непечатные символы. Визуальный анализ исходного файла и некоторые манипуляции в Python говорят о наличии табуляции в текстовых строках. Похоже, что на каком-то из этапов ETL что-то пошло не так. Конечно данная неприятность не является критической и с ней можно жить, но как-то не классно, когда в данных есть артефакты. Если есть ресурсы, то надо бы разобраться в причинах и починить. А пока очистим данные "руками". 


In [ ]:
# Чистим заголовки таблицы
sign_ups.columns = sign_ups.columns.str.replace('\t', '')
sign_ups.columns = sign_ups.columns.str.lower()

# Чистим сами данные
for i in sign_ups.columns:
  if sign_ups[i].dtype == 'object':
    # print(i, sign_ups[i].dtype)
    sign_ups[i] = sign_ups[i].str.strip('\t')
    zero = sign_ups[sign_ups[i]==''][i].count() #
    sign_ups[i] = sign_ups[i].replace('', np.nan)
    print('В переменной {} было произведено {} замен "пустоты" на NaN'.format(i, zero))

В переменной carrier было произведено 289 замен "пустоты" на NaN
В переменной city было произведено 447 замен "пустоты" на NaN
В переменной country было произведено 0 замен "пустоты" на NaN
В переменной device было произведено 0 замен "пустоты" на NaN
В переменной device_type было произведено 2752 замен "пустоты" на NaN
В переменной dma было произведено 5073 замен "пустоты" на NaN
В переменной ip_address было произведено 0 замен "пустоты" на NaN
В переменной language было произведено 0 замен "пустоты" на NaN
В переменной library было произведено 0 замен "пустоты" на NaN
В переменной os было произведено 0 замен "пустоты" на NaN
В переменной platform было произведено 0 замен "пустоты" на NaN
В переменной region было произведено 414 замен "пустоты" на NaN
В переменной gp:capmaign было произведено 2368 замен "пустоты" на NaN
В переменной gp:cohort_day было произведено 4954 замен "пустоты" на NaN
В переменной gp:cohort_month было произведено 4954 замен "пустоты" на NaN
В переменной gp:cohor

# Ответ на вопрос №1

Первоочередная задача - оценить результаты проведенного эксперимента. Поэтому в данном разделе я сфокусируюсь именно на ней. И буду обрабатывать/проверять/чистить только те данные, которые, как мне кажется, необходимы для ответа на поставленный вопрос. А все дополнительные изыскания по предоставленным данным сделаю в следующем разделе. 


Шаг №1 - объединить данные в единую таблицу.


In [ ]:
payments.set_index('email', inplace=True)
sign_ups.rename(columns={'gp:user_id': 'email'}, inplace=True)
sign_ups.set_index('email', inplace=True)

# df - не самое осмысленное название, но сделал это специально, для краткости. 
df = pd.merge(sign_ups, payments, how='left', left_index=True, right_index=True).reset_index()

Шаг №2 - разобраться с датами. Если с `payment_dt` все более-менее понятно, то вот с когортами есть вопрос. Являются ли данные в столбцах с названиями `gp:cohort_day` ,`gp:cohort_month`, `gp:cohort_week` и `gp:cohort_year` датами? И если да, то почему они так странно записаны? Зачем их разнесли на составляющие?


In [ ]:
for i in ['gp:cohort_day', 'gp:cohort_month', 'gp:cohort_year']:
  result = sorted(df[df[i].notnull()][i].unique().astype('int32'))
  print(result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[1, 10, 11, 12]
[2020, 2021, 2563, 2564]


Похоже, что это все же действительно даты. Остается вопрос - почему они хранятся в таком странном формате? И что за путешественники из будущего посещают приложение?) Исправим это и соединим дату когорты в дату.

In [ ]:
# Выясняем настоящий год когорты по месяцу
# df[(df['gp:cohort_year']=='2563')|(df['gp:cohort_year']=='2564')][['gp:cohort_year', 'gp:cohort_month']]

# Исправляем
df['gp:cohort_year'].replace('2564', '2021', inplace=True)
df['gp:cohort_year'].replace('2563', '2020', inplace=True)

df['cohort_date'] =  df['gp:cohort_year'] + '-' \
  + df['gp:cohort_month'] + '-' +df['gp:cohort_day'] 

# Меняем формат данных
df['cohort_date'] = pd.to_datetime(df['cohort_date'])
df['payment_dt'] = pd.to_datetime(df['payment_dt'])
df['payment_date'] = df['payment_dt'].dt.date.astype('datetime64')
df['payment_time'] = df['payment_dt'].dt.time


# Удалим лишнее
df.drop(['gp:cohort_year', 'gp:cohort_month', 'gp:cohort_day', 'gp:cohort_week', 'payment_dt'], axis=1, inplace=True)

Шаг №3 - изучим имеющийся временной диапазон по датам когорт и по датам оплат. 

(!) В предоставленных материалах ничего не сказано про когорты, по какому признаку они сформированы. Поэтому буду считать, что пользователь попадает в когорту когда первый раз скачивает/запускает приложение. 

In [ ]:
for i in ['cohort_date', 'payment_date']:
  print('{}: {} - {}'.format(i, df[i].min(), df[i].max()))

cohort_date: 2020-10-31 00:00:00 - 2021-01-12 00:00:00
payment_date: 2020-11-27 00:00:00 - 2021-01-10 00:00:00


Периоды совпадают. Точнее `cohort_date` чуть больше чем `payment_dt` и это хорошо. Значит можно будет посчитать конверсию из пользователей запустивших приложение в оплату на протяжении всего эксперимента ) 

Для дальнейшего удобства работы с данными распределим пользователей по группам эксперимента (А/В)

In [ ]:
df.loc[(df['payment_date']<'2020-12-19')|(df['cohort_date']<'2020-12-19'), 'group'] = 'A'
df.loc[(df['payment_date']>='2020-12-19')|(df['cohort_date']>='2020-12-19'), 'group'] = 'B'

Шаг №4 - оценим количество пользователей, наличие дублей и нет ли каких-то пользователей сразу в двух группах эксперимента. 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6382 entries, 0 to 6381
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   email            6382 non-null   object        
 1   amplitude_id     6382 non-null   int64         
 2   carrier          6093 non-null   object        
 3   city             5935 non-null   object        
 4   country          6382 non-null   object        
 5   device           6382 non-null   object        
 6   device_type      3630 non-null   object        
 7   dma              1309 non-null   object        
 8   ip_address       6382 non-null   object        
 9   language         6382 non-null   object        
 10  library          6382 non-null   object        
 11  os               6382 non-null   object        
 12  platform         6382 non-null   object        
 13  region           5968 non-null   object        
 14  start_version    6382 non-null   float64

In [ ]:
email = df['email'].nunique()
ampid = df['amplitude_id'].nunique()

print('unique_email: {}\n'
      'unique_amplitude_id: {}'.format(email, ampid))

unique_email: 6369
unique_amplitude_id: 6382


Достаточно большое количество пропусков в данных (это отдельный вопрос), но сейчас хочется понять что за дубликаты в `email`

In [ ]:
df[df['email'].duplicated(False)][['email', 'amplitude_id', 'gp:is_demo', 'group', 'device', 'ip_address']]

,email,amplitude_id,gp:is_demo,group,device,ip_address
45,578_t@yopmail.com,212850416735,True,NaN,Xiaomi Phone,188.227.114.94
46,578_t@yopmail.com,217203775394,True,B,Apple Simulator,188.227.114.94
47,578_t@yopmail.com,218598738085,True,NaN,google Google sdk_gphone_x86,188.227.114.94
48,578_t@yopmail.com,211539882783,True,A,Apple iPhone,188.227.114.94
49,578_t@yopmail.com,211545831056,False,NaN,Xiaomi Phone,188.227.114.94
487,anghelemanueldaniel@gmail.com,213014863129,True,NaN,xiaomi Xiaomi Redmi S2,67.218.232.202
488,anghelemanueldaniel@gmail.com,212286641374,True,NaN,CUBOT CUBOT CUBOT_POWER,67.218.232.202
978,brianbutcher@mail.com,223978734808,False,B,Apple iPhone,188.222.164.168
979,brianbutcher@mail.com,216150376351,False,B,Apple iPhone,79.69.235.50
1807,efmera@gmail.com,215516169471,True,A,Samsung Galaxy Phone,84.104.140.125


In [ ]:
target = df[df['email'].duplicated()]['email'].unique()
df = df[~df['email'].isin(target)]

Шаг №5 - проверим можно ли восстановить дату когорты по дате оплаты


In [ ]:
df[(df['payment_date'].notnull()&(df['cohort_date'].isnull()))]

,email,amplitude_id,carrier,city,country,device,device_type,dma,ip_address,language,library,os,platform,region,start_version,version,gp:capmaign,gp:is_demo,gp:media_source,amount,balance,cohort_date,payment_date,payment_time,group
70,a.cislaghi@outlook.it,222948629857,Iliad,Gallarate,Italy,OnePlus Phone,OnePlus 6T,NaN,151.30.133.127,Italian,amplitude-android/2.25.2,android 10,Android,Provincia di Varese,19.0,19.0,NaN,True,NaN,50.00,1000.00,NaT,2021-01-08,08:41:40,B
71,a.elmokhtari@gmail.com,212316647797,vodafone.de,Munich,Germany,HUAWEI HUAWEI AMN-LX9,NaN,NaN,109.41.192.23,German,amplitude-android/2.25.2,android 9,Android,Bavaria,14.0,16.0,NaN,False,NaN,66.85,1337.06,NaT,2020-12-03,02:23:20,A
84,aachleithner@gmail.com,212841711862,Orange,Krakow,Poland,Samsung Galaxy Phone,Samsung Galaxy S7 LTE-A,NaN,89.64.58.142,English,amplitude-android/2.25.2,android 8.0.0,Android,Lesser Poland,14.0,16.0,landing_email,False,landing_email,50.00,1000.04,NaT,2020-12-01,09:10:41,A
90,aangotzi@yahoo.it,221594783385,Iliad,Cagliari,Italy,realme realme RMX2170,NaN,NaN,151.56.52.124,Italian,amplitude-android/2.25.2,android 10,Android,Provincia di Cagliari,19.0,19.0,Cindicator Website QR,False,QR_code,50.00,1000.00,NaT,2021-01-05,13:49:10,B
361,allotherone@protonmail.com,215526315504,T-Mobile,Georgetown,United States,samsung samsung SM-G986U1,NaN,"Austin, TX",75.108.25.159,English,amplitude-android/2.25.2,android 11,Android,Texas,16.0,19.0,landing_email,False,landing_email,91.85,1836.93,NaT,2020-12-10,21:11:16,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6048,vincez.shenz@gmail.com,218521689431,Circles,Singapore,Singapore,Xiaomi Xiaomi M2007J3SY,NaN,NaN,103.252.203.229,English,amplitude-android/2.25.2,android 10,Android,NaN,18.0,19.0,NaN,False,NaN,50.00,1000.00,NaT,2020-12-22,09:36:38,B
6294,yordivd@outlook.com,222557096514,Orange B,Schilde,Belgium,Samsung Galaxy Phone,Samsung Galaxy S8,NaN,81.82.234.186,Dutch,amplitude-android/2.25.2,android 9,Android,Antwerp Province,19.0,19.0,NaN,False,NaN,50.00,1000.00,NaT,2021-01-08,07:22:09,B
6318,yylee.rey@gmail.com,214793186688,Singtel,Singapore,Singapore,Redmi Xiaomi Redmi Note 9 Pro,NaN,NaN,138.75.178.30,English,amplitude-android/2.25.2,android 10,Android,NaN,16.0,19.0,NaN,False,NaN,52.20,1044.00,NaT,2020-12-08,06:21:40,A
6325,zachdelane@gmail.com,211572594740,NaN,Burnsville,United States,Samsung Galaxy Phone,Samsung Galaxy S8,"Minneapolis-St. Paul, MN",174.250.64.96,English,amplitude-android/2.25.2,android 9,Android,Minnesota,13.0,19.0,landing_email,False,landing_email,54.85,1097.00,NaT,2020-11-27,19:22:20,A


 У 54% пользователей с `payment_dt` нет данных в `cohort_date` и это странно. Получается, что эти пользователи не попали ни в какую когорту для аналитики, но они есть в платежках. Я сделаю очень грубое допущение, что дата совершения платежа (в данном случае) является и датой первого запуска/регистрации приложения. А вообще, хорошо бы разобраться в причинах появления таких пропусков. 

In [ ]:
# Восстановим cohort_date из payment_date там, где это возможно
df.loc[(df['payment_date'].notnull()&(df['cohort_date'].isnull())), 'cohort_date'] = df['payment_date']

In [ ]:
# Считаем конверсию
conversion = (df
 .pivot_table(index='group', aggfunc={'cohort_date':'count', 'payment_date':'count'})
 .rename(columns={'cohort_date':'sign-up', 'payment_date':'purchase'})).T

In [ ]:
# Рисуем конверсию
def my_funnel(df, title):
    """
    Функция принимает df и название графика. Воронка строится по событиям, которые должны являться индексами в указанном df.
    Если указанный df содержит несколько переменных, то воронка будет построена по всем данным df
    """
    
    fig = go.Figure()

    for name in df.columns:
        fig.add_trace(go.Funnel(
            name = name,
            y = df.index,
            x = df[name],
            textposition = 'inside',
            textinfo = 'value+percent previous'))
        
    fig.update_layout(title=title,
                      margin=dict(l=0, r=0))

    fig.show()

my_funnel(conversion, 'Конверсия пользователей')

Складывается ощущение, что пользователи группы B (после изменения воронки) конвертируются лучше. Пожалуй стоит проверить наличие/отсутствие статистической значимости в цифрах. Вдруг это просто случайность? 

Для этого используем Z-тест (z-критерий Фишера).  За нулевую гипотезу буду принимать, что в разницы в конверсии нет.

In [ ]:
def result_ztest(df):
    alpha = 0.05 
    
    i = 0
    while i < 1: 
        print('Шаг воронки: {} -> {}:'.format(df.index[i], df.index[i+1]))

        # исходные данные
        first_step = np.array([df.iloc[i][0], df.iloc[i][1]])
        next_step = np.array([df.iloc[i+1][0], df.iloc[i+1][1]])

        # пропорция успехов в первой группе:
        p1 = next_step[0]/first_step[0]

        # пропорция успехов во второй группе:
        p2 = next_step[1]/first_step[1]

        # пропорция успехов в комбинированном датасете:
        p_combined = (next_step[0] + next_step[1]) / (first_step[0] + first_step[1])

        # разница пропорций в датасетах
        difference = p1 - p2

        # считаем статистику в ст.отклонениях стандартного нормального распределения
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/first_step[0] + 1/first_step[1]))

        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
        distr = st.norm(0, 1)

        p_value = (1 - distr.cdf(abs(z_value))) * 2

        print('p-значение: ', p_value)
        if (p_value < alpha):
            print("Отвергаем нулевую гипотезу, есть статистически значимая разница")
        else:
            print("Не получилось отвергнуть нулевую гипотезу, нет статистически значимой разницы")

        print('---'*30)
        print()
        i = i + 1

result_ztest(conversion)

Шаг воронки: sign-up -> purchase:
p-значение:  0.00014885414194609758
Отвергаем нулевую гипотезу, есть статистически значимая разница
------------------------------------------------------------------------------------------



Если верить полученным результатом, то с вероятностью в 95% изменение шагов воронки дает прирост конверсии. И вроде все хорошо, эксперимент можно считать удачным, но хочется посмотреть и на другие метрики в разрезе групп пользователей. Например на выручку. 

Я надеюсь, что правильно трактую смысл переменной  `gp:is_demo`. 
- True - пользователь находится на demo аккаунте. 
- False - пользователь использует не demo аккаунт => `amount` это суммы, которые получил сервис за свою работу. 


In [ ]:
(df
 .pivot_table(index=['gp:is_demo', 'group'], aggfunc={'amount':'sum', 'payment_date':'count'})
 .rename(columns={'amount':'total_amount', 'payment_date':'total_user'})
)

total_amount  total_user
gp:is_demo group                          
False      A          16171.13          95
           B          10237.02          96
True       A            204.76           3
           B           3798.30          43

Хм, получается, что при сопоставимом количестве платящих пользователей больше денег приносила группа А. В таком разрезе трудно признать эксперимент по изменению воронки успешным. Если бы в предоставленных данных была информация о стоимости привлечения пользователя и LTV. То можно было бы рассчитать ROI и более взвешенно (как мне кажется) оценить успех/провал эксперимента.  


# Ответ на вопрос №2



*Если возможно, определите наиболее эффективные каналы привлечения и другие важные параметры юнит-экономики продукта, которые заметите на основе данных из п.1.*

## Наиболее эффективные каналы привлечения


In [ ]:
df['gp:capmaign'].value_counts()

29889                           1061
29877                            363
29885                            306
30397                            284
release_capmaign                 259
landing_email                    229
29931                            198
Cindicator Website QR            181
CND_App-Stoic                    129
Android Store Link               127
29929                            125
English AMA                      100
Apple Store Website Link          90
29873                             87
Blog                              61
US - Broad                        60
30398                             42
29882                             42
29891                             36
Tweets                            31
US - Keywords                     26
The Crypto Hamster                24
Other - Keywords                  22
Stoic portfolio - 1516880502      14
None                              14
twitter                           10
Tik Tok The Wolf of Bitcoins       9
m

In [ ]:
df['gp:media_source'].value_counts()

bidease_int           2546
landing                259
landing_email          229
Cindicator Website     217
QR_code                181
Apple Search Ads       170
Cross_sale             142
Binance                100
Blog                    61
Social_twitter          41
Telegram Promo          24
mail.ru_int             13
Tik Tok                  9
blog                     6
Social_facebook          4
blockfolio               4
None                     1
cointraffic              1
medium                   1
Name: gp:media_source, dtype: int64

Вау. Самые массовые кампании (по количеству пользователей) носят самы загадочные названия. Надеюсь, что сотрудники которые могут сразу идентифицировать ту информацию еще работают в штате )

С названием источников ситуация сильно лучше, они осмысленные )

In [ ]:
# Не самое красивое решение )  
left = df.pivot_table(index='gp:media_source', aggfunc={'email':'count'}).reset_index()
right = df.pivot_table(index='gp:media_source', columns='gp:is_demo', aggfunc={'email':'count'}).rename(columns={'False':'demo_false', 'True':'demo_true'})
right.columns = right.columns.droplevel()

result = left.join(right, on='gp:media_source')
result['demo_false_conv'] = result['demo_false'] / result['email'] * 100
result.sort_values(by='demo_false_conv', ascending=False)

,gp:media_source,email,demo_false,demo_true,demo_false_conv
5,None,1,1.0,NaN,100.000000
16,landing_email,229,47.0,154.0,20.524017
2,Blog,61,11.0,36.0,18.032787
15,landing,259,45.0,175.0,17.374517
13,blog,6,1.0,3.0,16.666667
6,QR_code,181,23.0,134.0,12.707182
9,Telegram Promo,24,3.0,14.0,12.500000
3,Cindicator Website,217,12.0,187.0,5.529954
8,Social_twitter,41,2.0,31.0,4.878049
4,Cross_sale,142,4.0,119.0,2.816901


Итак, 
- Источник с 100% конверсией скорее всего следствие каких-то экспериментов/тестов 
- `landing_email` - предположу, что это рассылка по пользователям оставившим email на лендинге. И конверсия достаточно хорошая. Значит на лендинг приводится целевой трафик и потом с ним идет дополнительная работа через рассылки. 
- `Blog` - здесь вроде тоже все понятно, целевой контент в блоге => целевые посетители => целевые действия
- `landing` - видимо прямая продажа сразу через лендинг. Вопросов нет. 

А вот есть вопросы к следующему:
- разное написание одинаковых источников это не круто, но это мелочи. 
- `bidease_int` - если я правильно идентифицировал источник, то это Mobile Programmatic Platform BIDEASE и с конверсией платящих пользователей пока все очень и очень плохо. Текущий набор данных не позволяет (или я не увидел) отследить конверсию из demo_true в demo_false, поэтому при оценке "в лоб" данный источник проигрывает всем остальным. Интересно сколько бюджета было потрачено на эту активность?


## Другие важные параметры юнит-экономики продукта

DAU, WAU, MAU


In [ ]:
dauwaumau = df[df['cohort_date'].notnull()][['email', 'cohort_date']]
dauwaumau['cohort_year'] = dauwaumau['cohort_date'].dt.year
dauwaumau['cohort_month'] = dauwaumau['cohort_date'].astype('datetime64[M]')
dauwaumau['cohort_week'] = dauwaumau['cohort_date'].dt.week

DAU = int(dauwaumau
          .reset_index()
          .groupby('cohort_date').agg({'email':'nunique'}).mean())
WAU = int(dauwaumau
          .reset_index()
          .groupby(['cohort_year', 'cohort_week']).agg({'email':'nunique'}).mean())
MAU = int(dauwaumau
          .reset_index()
          .groupby(['cohort_year', 'cohort_month']).agg({'email':'nunique'}).mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



In [ ]:
fig = go.Figure(
    data=[go.Bar(
        y=[DAU, WAU, MAU],
        x=['DAU','WAU','MAU'])])

fig.update_layout(
    title={
        'text': 'Пользовательская активность: DAU, WAU, MAU',
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title='Количество уникальных пользователей',)

fig.show()

Вроде все достаточно логично и есть рост от DAU к WAU и к MAU. Но очевидно, что он не линейный и есть потери пользователей. Посмотрим на дополнительную метрику **sticky factor**

In [ ]:
sticky_wau = DAU / WAU
sticky_mau = DAU / MAU

print('sticky factor WAU: {:.2%}'.format(sticky_wau))
print('sticky factor MAU: {:.2%}'.format(sticky_mau))

sticky factor WAU: 17.65%
sticky factor MAU: 5.41%


Получается, что регулярность посещения сервиса уникальными пользователями составляет:
* 17.65% по неделям
* 5.41% по месяцам
Думаю, что было бы полезно сравнить эти значения с продуктами конкурентов (если есть такая возможность), а также с отраслевыми стандартами. Возможно, что стоит озадачиться увеличением данных значений и стимулировать аудиторию возвращаться как можно чаще.

### Buyers

In [ ]:
left = (df
        .pivot_table(index='cohort_date', aggfunc={'email':'nunique', 'payment_date':'count'})
        .rename(columns={'email':'total_user', 'payment_date':'payment_user'}))

right = (df[df['gp:is_demo']=='True']
         .pivot_table(index='cohort_date', aggfunc={'payment_date':'count'})
         .rename(columns={'payment_date':'money_user'}))

result = left.join(right)

In [ ]:
result.sum()

total_user      1555.0
payment_user     238.0
money_user        46.0
dtype: float64

- total_user - все пользователи у которых есть дата попадания в когорту
- payment_user - все пользователи по которым есть запись о совершении платежа (без учета demo-аккаунт или нет)
-  money_user - все пользователи по которым есть запись о совершении платежа и у них не demo-аккаунт 

Я не стал учитывать в данных расчетах пользователей без даты когорты/оплаты. К чистоте данных достаточно много вопросов. Не исключено, что пользователи без дат могут быть вообще их других временных периодов. А может и нет, но проверить это сейчас нельзя )

In [ ]:
def my_scatter(data, title, y_title):
    '''
    Функция строит график по всем столбцам переданного DataFrame (data). 
    Важно, чтобы индекс передаваемого DataFrame был в формате даты. 
    title - указать название будущего графика
    '''
    fig = go.Figure()

    for i in data.columns:
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data[i],
            name=i,
            mode='lines'
        ))

    fig.update_yaxes(title=y_title)

    fig.update_layout(
        title=title
    )
    
    fig.show()

my_scatter(result, 'Динамика пользователей', 'Количество пользователей ')

Похоже, что в начале января начали принимать активные действия по росту платящих пользователей. 

In [ ]:
df.loc[(df['gp:is_demo']=='True')&(df['payment_date'].notnull()), 'money'] = 'yes'

### ARPU

In [ ]:
df[df['money']=='yes'].groupby('payment_date')['email'].count()

payment_date
2020-11-27    1
2020-12-03    1
2020-12-09    1
2020-12-20    1
2020-12-23    1
2020-12-25    1
2020-12-26    1
2020-12-28    2
2020-12-29    1
2020-12-31    2
2021-01-02    1
2021-01-03    2
2021-01-04    5
2021-01-05    2
2021-01-06    3
2021-01-07    6
2021-01-08    3
2021-01-09    7
2021-01-10    5
Name: email, dtype: int64

По хорошему ARPU надо считать за месяц. Но данных по оплатам так мало, что это все выглядит как-то грустно. Поэтому посчитаю его за весь временной период. 

In [ ]:
df[df['money']=='yes']['amount'].sum() / df[df['money']=='yes']['email'].count()

87.02304347826087

**Marketing cost**, **CPA**, **CAC**, **LTV**, **Retention**, **Margin** - все эти чудесные метрики нельзя сейчас рассчитать, так как нет исходных даных. Ну или я не вижу как это сделать, но тогда это полный провал )

# Ответ на вопрос №3

Попробую определить потенциальные проблемы зоны в механизмах сбора данных:
- непечатные символы в выгрузках это проблема. Могут вылезти в самый неподходящий момент и сломать всю автоматизацию. Хорошо бы проверить все этапы ETL и сохранять в БД только данные, без табуляций и пробелов. 
- предоставленные данные содержат большое количество пропусков. Причем порой в очень странных местах. `gp:cohort_day` ,`gp:cohort_month` , `gp:cohort_year` - почему большая часть пользователей не имеет данные о дате? Это ошибка сбора или хранения? И, честно говоря, пока не очень понятно зачем дату хранить в разных метриках? Почему не хранить все в типе дата? 
- 'gp:cohort_year' как в автоматическую выгрузку из системы аналитики попали 2563, 2564? Ручной ввод? Ошибка? Где и почему?
- `gp:is_demo` пожалуй можно назвать одной из ключевых метрик. Важно же понимать статус пользователя и когда он стал платящим? Но почему так много NaN? Неужели аналитика настроена так плохо, что данные не собираются? 
- `gp:capmaign` и `gp:media_source` и здесь опять пропуски, пропуски, пропуски. Откуда появились все эти пользователи без источников? Кто они? 
- `dma`, `device_type`, `city` пропуски в данных из-за оператора связи или что-то сломалось на этапе сбора и хранения информации? 
- Почему пользователи с одним email получают разные `amplitude_id`? Их оказалось всего 13, но ведь если это какой-то баг, то завтра таких пользователей может стать 130
- У 54% пользователей с `payment_dt` нет данных в `cohort_date `и это странно. Получается, что эти пользователи не попали ни в какую когорту для аналитики, но они есть в платежках.
- 'gp:capmaign' нужно прививать культуру нормального названия источников/кампаний )

Можно было еще больше покопаться в даных и поискать взаимосвязь между конверсией и типом телефона/версией операционки/версией установленного приложения/регионом/городом. Но мне кажется, что общий уровень работы получится оценить и по уже готовому отчету )  